# <font color="#f47641">cread detile csv</font>  
###  <font color="#DAC9A6"># deal with normalization </font>  



In [ ]:
import pandas as pd
df = pd.read_csv("../")

In [ ]:
import IPython.display as ipd

import numpy
sr = 22050 # sample rate
T = 0.2    # seconds
t = numpy.linspace(0, T, int(T*sr), endpoint=False) # time variable
x = 0.5*numpy.sin(2*numpy.pi*490*t)                # pure sine wave at 440 Hz
ipd.Audio(x, rate=sr,autoplay=True) # load a NumPy array

In [ ]:
import glob
import re
import pandas as pd
from sklearn import preprocessing 
import matplotlib.pyplot as plt

count = 0
is_normal=["normal","abnormal"]

for a in is_normal:
    df = pd.DataFrame(columns=['batch','sensor','value','status'])
    df["value"]=df["value"].astype("float")
    for filename in (glob.glob('ecg/'+a+'/*')):
        file = pd.read_csv(filename,sep="   ",header=None)
        batch=re.search("8.+?(?=\.)",filename)
        sensor=re.search("(?<=\.)(.*)",filename)
        status=a
        if (sensor.group(0)!="ann" and sensor.group(0)!="box"):
            value = file[1].values
            #normalization
            value = preprocessing.minmax_scale(value,feature_range=(0,1))
            df.loc[count]=[batch.group(0),sensor.group(0),value,status]
            
        count+=1

    df.to_json("ecg_"+a+".json")

# #write csv to let the value type in array

print("done")

# <font color="#f47641">create train and test dataset csv</font>  
###  <font color="#DAC9A6"> # if want change train and test set change seed </font>  
###  <font color="#DAC9A6"> # separate data to train test validation        </font>  



In [ ]:
import pandas as pd
import random 
import numpy as np

def dataset(is_whole):
    if is_whole==True:
        np.random.seed(1)
        df_normal = pd.read_json("ecg_normal.json")
        df_abnormal = pd.read_json("ecg_abnormal.json")
        np.random.seed(1)
        normal_train =np.random.choice(df_normal.batch.unique(),93,replace=False)
        abnormal_train =np.random.choice(df_abnormal.batch.unique(),47,replace=False)

        df_normal_train=df_normal[df_normal.batch.isin(normal_train)]
        df_normal_left=df_normal[~df_normal.batch.isin(normal_train)]
        normal_validation =np.random.choice(df_normal_left.batch.unique(),20,replace=False)
        df_normal_validation=df_normal_left[df_normal_left.batch.isin(normal_validation)]
        df_normal_test=df_normal_left[~df_normal_left.batch.isin(normal_validation)]

        df_abnormal_train=df_abnormal[df_abnormal.batch.isin(abnormal_train)]
        df_abnormal_left=df_abnormal[~df_abnormal.batch.isin(abnormal_train)]
        abnormal_validation =np.random.choice(df_abnormal_left.batch.unique(),10,replace=False)
        df_abnormal_validation=df_abnormal_left[df_abnormal_left.batch.isin(abnormal_validation)]
        df_abnormal_test=df_abnormal_left[~df_abnormal_left.batch.isin(abnormal_validation)]


        df_normal_train.to_json("ecg_normal_train_all.json")
        df_abnormal_train.to_json("ecg_abnormal_train_all.json")
        df_normal_validation.to_json("ecg_normal_validation_all.json")
        df_abnormal_validation.to_json("ecg_abnormal_validation_all.json")
        df_normal_test.to_json("ecg_normal_test_all.json")
        df_abnormal_test.to_json("ecg_abnormal_test_all.json")
        
    else: 
        np.random.seed(1)
        df_normal = pd.read_json("ecg_normal.json")
        normal_sample =np.random.choice(df_normal.batch.unique(),70,replace=False)
        df_normal_sample=df_normal[df_normal.batch.isin(normal_sample)]

        df_abnormal = pd.read_json("ecg_abnormal.json")

        np.random.seed(1)
        normal_train =np.random.choice(df_normal_sample.batch.unique(),50,replace=False)
        abnormal_train =np.random.choice(df_abnormal.batch.unique(),47,replace=False)

        df_normal_train=df_normal_sample[df_normal_sample.batch.isin(normal_train)]
        df_normal_left=df_normal_sample[~df_normal_sample.batch.isin(normal_train)]
        normal_validation =np.random.choice(df_normal_left.batch.unique(),10,replace=False)
        df_normal_validation=df_normal_left[df_normal_left.batch.isin(normal_validation)]
        df_normal_test=df_normal_left[~df_normal_left.batch.isin(normal_validation)]

        df_abnormal_train=df_abnormal[df_abnormal.batch.isin(abnormal_train)]
        df_abnormal_left=df_abnormal[~df_abnormal.batch.isin(abnormal_train)]
        abnormal_validation =np.random.choice(df_abnormal_left.batch.unique(),10,replace=False)
        df_abnormal_validation=df_abnormal_left[df_abnormal_left.batch.isin(abnormal_validation)]
        df_abnormal_test=df_abnormal_left[~df_abnormal_left.batch.isin(abnormal_validation)]


        df_normal_train.to_json("ecg_normal_train.json")
        df_abnormal_train.to_json("ecg_abnormal_train.json")
        df_normal_validation.to_json("ecg_normal_validation.json")
        df_abnormal_validation.to_json("ecg_abnormal_validation.json")
        df_normal_test.to_json("ecg_normal_test.json")
        df_abnormal_test.to_json("ecg_abnormal_test.json")



In [ ]:
dataset(True)
dataset(False)

# <font color="#f47641">create crossvalidation dataset csv</font>  
###  <font color="#DAC9A6"> # if want change train and test set change seed </font>  
###  <font color="#DAC9A6"> # separate data to 5-fold crossvalidation        </font>  



In [ ]:
def dataset_cross(is_whole):
    import pandas as pd
    import random 
    import numpy as np
    if is_whole==True:
        np.random.seed(1)
        df_normal = pd.read_json("ecg_normal.json")
        df_abnormal = pd.read_json("ecg_abnormal.json")
        np.random.seed(1)

        normal_coross_1 =np.random.choice(df_normal.batch.unique(),27,replace=False)
        df_normal_cross_1=df_normal[df_normal.batch.isin(normal_coross_1)]
        df_normal_cross_1["cross"]=1
        df_normal_left=df_normal[~df_normal.batch.isin(normal_coross_1)]

        normal_coross_2 =np.random.choice(df_normal_left.batch.unique(),26,replace=False)
        df_normal_cross_2=df_normal_left[df_normal_left.batch.isin(normal_coross_2)]
        df_normal_cross_2["cross"]=2
        df_normal_left=df_normal_left[~df_normal_left.batch.isin(normal_coross_2)]

        normal_coross_3 =np.random.choice(df_normal_left.batch.unique(),27,replace=False)
        df_normal_cross_3=df_normal_left[df_normal_left.batch.isin(normal_coross_3)]
        df_normal_cross_3["cross"]=3
        df_normal_left=df_normal_left[~df_normal_left.batch.isin(normal_coross_3)]

        normal_coross_4 =np.random.choice(df_normal_left.batch.unique(),26,replace=False)
        df_normal_cross_4=df_normal_left[df_normal_left.batch.isin(normal_coross_4)]
        df_normal_cross_4["cross"]=4
        df_normal_cross_5=df_normal_left[~df_normal_left.batch.isin(normal_coross_4)]
        df_normal_cross_5["cross"]=5

        abnormal_coross_1 =np.random.choice(df_abnormal.batch.unique(),13,replace=False)
        df_abnormal_cross_1=df_abnormal[df_abnormal.batch.isin(abnormal_coross_1)]
        df_abnormal_cross_1["cross"]=1
        df_abnormal_left=df_abnormal[~df_abnormal.batch.isin(abnormal_coross_1)]

        abnormal_coross_2 =np.random.choice(df_abnormal_left.batch.unique(),14,replace=False)
        df_abnormal_cross_2=df_abnormal_left[df_abnormal_left.batch.isin(abnormal_coross_2)]
        df_abnormal_cross_2["cross"]=2
        df_abnormal_left=df_abnormal_left[~df_abnormal_left.batch.isin(abnormal_coross_2)]

        abnormal_coross_3 =np.random.choice(df_abnormal_left.batch.unique(),13,replace=False)
        df_abnormal_cross_3=df_abnormal_left[df_abnormal_left.batch.isin(abnormal_coross_3)]
        df_abnormal_cross_3["cross"]=3
        df_abnormal_left=df_abnormal_left[~df_abnormal_left.batch.isin(abnormal_coross_3)]

        abnormal_coross_4 =np.random.choice(df_abnormal_left.batch.unique(),14,replace=False)
        df_abnormal_cross_4=df_abnormal_left[df_abnormal_left.batch.isin(abnormal_coross_4)]
        df_abnormal_cross_4["cross"]=4
        df_abnormal_cross_5=df_abnormal_left[~df_abnormal_left.batch.isin(abnormal_coross_4)]
        df_abnormal_cross_5["cross"]=5


        df_normal_cross_1.to_json("ecg_normal_all_cross_1.json")
        df_normal_cross_2.to_json("ecg_normal_all_cross_2.json")
        df_normal_cross_3.to_json("ecg_normal_all_cross_3.json")
        df_normal_cross_4.to_json("ecg_normal_all_cross_4.json")
        df_normal_cross_5.to_json("ecg_normal_all_cross_5.json")

        df_abnormal_cross_1.to_json("ecg_abnormal_all_cross_1.json")
        df_abnormal_cross_2.to_json("ecg_abnormal_all_cross_2.json")
        df_abnormal_cross_3.to_json("ecg_abnormal_all_cross_3.json")
        df_abnormal_cross_4.to_json("ecg_abnormal_all_cross_4.json")
        df_abnormal_cross_5.to_json("ecg_abnormal_all_cross_5.json")

        print(len(df_normal_cross_1),len(df_normal_cross_2),len(df_normal_cross_3),len(df_normal_cross_4),len(df_normal_cross_5))
        print(len(df_abnormal_cross_1),len(df_abnormal_cross_2),len(df_abnormal_cross_3),len(df_abnormal_cross_4),len(df_abnormal_cross_5))
    else:
        np.random.seed(1)
        df_normal = pd.read_json("ecg_normal.json")
        normal_sample =np.random.choice(df_normal.batch.unique(),70,replace=False)
        df_normal_sample=df_normal[df_normal.batch.isin(normal_sample)]
        df_abnormal = pd.read_json("ecg_abnormal.json")
        np.random.seed(1)

        normal_coross_1 =np.random.choice(df_normal_sample.batch.unique(),14,replace=False)
        df_normal_cross_1=df_normal_sample[df_normal_sample.batch.isin(normal_coross_1)]
        df_normal_cross_1["cross"]=1
        df_normal_left=df_normal_sample[~df_normal_sample.batch.isin(normal_coross_1)]

        normal_coross_2 =np.random.choice(df_normal_left.batch.unique(),14,replace=False)
        df_normal_cross_2=df_normal_left[df_normal_left.batch.isin(normal_coross_2)]
        df_normal_cross_2["cross"]=2
        df_normal_left=df_normal_left[~df_normal_left.batch.isin(normal_coross_2)]

        normal_coross_3 =np.random.choice(df_normal_left.batch.unique(),14,replace=False)
        df_normal_cross_3=df_normal_left[df_normal_left.batch.isin(normal_coross_3)]
        df_normal_cross_3["cross"]=3
        df_normal_left=df_normal_left[~df_normal_left.batch.isin(normal_coross_3)]

        normal_coross_4 =np.random.choice(df_normal_left.batch.unique(),14,replace=False)
        df_normal_cross_4=df_normal_left[df_normal_left.batch.isin(normal_coross_4)]
        df_normal_cross_4["cross"]=4
        df_normal_cross_5=df_normal_left[~df_normal_left.batch.isin(normal_coross_4)]
        df_normal_cross_5["cross"]=5

        abnormal_coross_1 =np.random.choice(df_abnormal.batch.unique(),13,replace=False)
        df_abnormal_cross_1=df_abnormal[df_abnormal.batch.isin(abnormal_coross_1)]
        df_abnormal_cross_1["cross"]=1
        df_abnormal_left=df_abnormal[~df_abnormal.batch.isin(abnormal_coross_1)]

        abnormal_coross_2 =np.random.choice(df_abnormal_left.batch.unique(),14,replace=False)
        df_abnormal_cross_2=df_abnormal_left[df_abnormal_left.batch.isin(abnormal_coross_2)]
        df_abnormal_cross_2["cross"]=2
        df_abnormal_left=df_abnormal_left[~df_abnormal_left.batch.isin(abnormal_coross_2)]

        abnormal_coross_3 =np.random.choice(df_abnormal_left.batch.unique(),13,replace=False)
        df_abnormal_cross_3=df_abnormal_left[df_abnormal_left.batch.isin(abnormal_coross_3)]
        df_abnormal_cross_3["cross"]=3
        df_abnormal_left=df_abnormal_left[~df_abnormal_left.batch.isin(abnormal_coross_3)]

        abnormal_coross_4 =np.random.choice(df_abnormal_left.batch.unique(),14,replace=False)
        df_abnormal_cross_4=df_abnormal_left[df_abnormal_left.batch.isin(abnormal_coross_4)]
        df_abnormal_cross_4["cross"]=4
        df_abnormal_cross_5=df_abnormal_left[~df_abnormal_left.batch.isin(abnormal_coross_4)]
        df_abnormal_cross_5["cross"]=5


        df_normal_cross_1.to_json("ecg_normal_cross_1.json")
        df_normal_cross_2.to_json("ecg_normal_cross_2.json")
        df_normal_cross_3.to_json("ecg_normal_cross_3.json")
        df_normal_cross_4.to_json("ecg_normal_cross_4.json")
        df_normal_cross_5.to_json("ecg_normal_cross_5.json")

        df_abnormal_cross_1.to_json("ecg_abnormal_cross_1.json")
        df_abnormal_cross_2.to_json("ecg_abnormal_cross_2.json")
        df_abnormal_cross_3.to_json("ecg_abnormal_cross_3.json")
        df_abnormal_cross_4.to_json("ecg_abnormal_cross_4.json")
        df_abnormal_cross_5.to_json("ecg_abnormal_cross_5.json")

        print(len(df_normal_cross_1),len(df_normal_cross_2),len(df_normal_cross_3),len(df_normal_cross_4),len(df_normal_cross_5))
        print(len(df_abnormal_cross_1),len(df_abnormal_cross_2),len(df_abnormal_cross_3),len(df_abnormal_cross_4),len(df_abnormal_cross_5))
        

In [ ]:
dataset_cross(True)
dataset_cross(False)

# <font color="#f47641">Represent to image</font>  
###  <font color="#DAC9A6"> # Write in function just need call method </font>  
###  <font color="#DAC9A6"> # is_whole : decide call all datasate or sample datasat </font>  
###  <font color="#DAC9A6"> # is_cross : decide call crossvalidation dataset or not </font>  

###  <font color="#DAC9A6"> # method : "mtf","gasf","gadf","recurrence_plots" </font>  





In [ ]:
def transformation(is_whole,is_cross,method):
    from pyts.visualization import plot_gasf,plot_gadf,plot_mtf,plot_recurrence_plots
    import matplotlib.pyplot as plt
    import os
    import pandas as pd
    import numpy as np
    from PIL import Image

    
    if is_cross==False:
        is_train=["train","test","validation"]
        is_normal=["normal","abnormal"]
        if is_whole==True:
            path="_all"
        else:
            path="" 
        for a in is_train:
            for b in is_normal:
                x= pd.read_json("ecg_"+b+"_"+a+path+".json")
                for i in range(len(x)):
                    x_time=np.array(x.iloc[:,3].values[i])
                    x_class= str(x.iloc[:,0].values[i])+'_'+str(x.iloc[:,1].values[i])
                    if method.lower() != "recurrence_plots":

                        locals()["plot_"+method.lower()](x_time,
                                                 cmap="gray",
                                                 image_size=39, 
                                                 output_file="ecg_img"+path+"/"+method+"/"+a+"/"+b+"/"+x_class+".png")
                    else:
                        locals()["plot_"+method.lower()](x_time,
                                                 output_file="ecg_img"+path+"/"+method+"/"+a+"/"+b+"/"+x_class+".png")

                for i in range(len(x)):
                    x_time=np.array(x.iloc[:,3].values[i])
                    x_class= str(x.iloc[:,0].values[i])+'_'+str(x.iloc[:,1].values[i])
                    img = Image.open("ecg_img"+path+"/"+method+"/"+a+"/"+b+"/"+x_class+".png")
                    img=img.crop((20, 3, 237, 220))
                    img.thumbnail((128,128))  #resize
                    img.save("ecg_img"+path+"/"+method+"/"+a+"/"+b+"/"+x_class+".png")
    else:
        if is_whole==True:
            path="_all"
        else:
            path=""
        cross=["1","2","3","4","5"]
        is_normal=["normal","abnormal"]
        for a in cross:
            for b in is_normal:
                x= pd.read_json("ecg_"+b+path+"_cross_"+a+".json")
                for i in range(len(x)):
                    x_time=np.array(x.iloc[:,4].values[i])
                    x_class= str(x.iloc[:,0].values[i])+'_'+str(x.iloc[:,2].values[i])
                    if method.lower() != "recurrence_plots":

                        locals()["plot_"+method.lower()](x_time,
                                                 cmap="gray",
                                                 image_size=39, 
                                                 output_file="ecg_img_cross"+path+"/"+method+"/"+a+"/"+b+"/"+x_class+".png")
                    else:
                        locals()["plot_"+method.lower()](x_time,
                                                 output_file="ecg_img_cross"+path+"/"+method+"/"+a+"/"+b+"/"+x_class+".png")

                for i in range(len(x)):
                    x_time=np.array(x.iloc[:,4].values[i])
                    x_class= str(x.iloc[:,0].values[i])+'_'+str(x.iloc[:,2].values[i])
                    img = Image.open("ecg_img_cross"+path+"/"+method+"/"+a+"/"+b+"/"+x_class+".png")
                    img=img.crop((20, 3, 237, 220))
                    img.thumbnail((128,128))  #resize
                    img.save("ecg_img_cross"+path+"/"+method+"/"+a+"/"+b+"/"+x_class+".png")


In [ ]:
transformation(True,True,"gasf")
transformation(True,True,"gadf")
transformation(True,True,"mtf")
transformation(True,True,"recurrence_plots")
print("done whole=true cross=true")

transformation(True,False,"gasf")
transformation(True,False,"gadf")
transformation(True,False,"mtf")
transformation(True,False,"recurrence_plots")
print("done whole=true cross=False")

In [ ]:
import IPython.display as ipd

import numpy
sr = 22050 # sample rate
T = 0.2    # seconds
t = numpy.linspace(0, T, int(T*sr), endpoint=False) # time variable
x = 0.5*numpy.sin(2*numpy.pi*490*t)                # pure sine wave at 440 Hz
ipd.Audio(x, rate=sr,autoplay=True) # load a NumPy array

# <font color="#f47641">create dataset & dataloader</font>  

## six channel

In [ ]:
from torch.utils.data import Dataset, DataLoader
class Dataset(Dataset):
    def __init__(self,train_x,train_y):
        self.train_x=train_x
        self.train_y=train_y
        self.data_len = len(train_x)

    def __getitem__(self, index):

        train_x=self.train_x[index]
        train_y=self.train_y[index]
        return (train_x, train_y)
    def __len__(self):
        return self.data_len
    
class dataset_dataloader:
    def __init__ (self,is_whole,is_cross,method,cross_number_for_test=1,batchsize=24,numworkers=2):
        if is_cross==False:
            import pandas as pd
            import numpy as np
            from PIL import Image
            from pyts.visualization import plot_gasf
            import matplotlib.pyplot as plt
            import torch
            from torch.autograd import Variable

            if is_whole==True:
                whole="_all"
            else:
                whole=""

            normal   = pd.read_json("ecg_normal_train"+whole+".json")
            abnormal = pd.read_json("ecg_abnormal_train"+whole+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            def pil_loader(path):
                with open(path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('L')
            lst = list()
            count=0
            lst2=list()
            y=list()

            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="ecg_img"+whole+"/"+method+"/train/abnormal/"
                else:
                    path="ecg_img"+whole+"/"+method+"/train/normal/"

                for j in x.sensor.unique():
                    img=pil_loader(path+str(i)+"_"+str(j)+".png")
                    lst.append(np.array(img)/255)
                    
                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     count+=1
            #     if count==1:
            #         break
            train_x = torch.from_numpy(np.array(lst2)).float()
            train_y = torch.LongTensor(np.array(y))

            train_data = Dataset(train_x,train_y)
            train_loader = DataLoader(dataset=train_data,
                                      batch_size=24,
                                      shuffle=True,
                                      num_workers=2)
            self.train_x=train_x
            self.train_y=train_y
            self.train_loader=train_loader

            #test data set
            normal   = pd.read_json("ecg_normal_validation"+whole+".json")
            abnormal = pd.read_json("ecg_abnormal_validation"+whole+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            # print(x)
            lst = list()
            count=0
            lst2=list()
            y=list()
            path=""
            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="ecg_img"+whole+"/"+method+"/validation/abnormal/"
                else:
                    path="ecg_mg"+whole+"/"+method+"/validation/normal/"

                for j in x.sensor.unique():
                    img=pil_loader(path+str(i)+"_"+str(j)+".png")
                    lst.append(np.array(img)/255)
                    
                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     if count==1:
            #         break
            test_x = Variable(torch.from_numpy(np.array(lst2)).float()).cuda()
            test_y = torch.LongTensor(np.array(y)).cuda()
            self.test_x=test_x
            self.test_y=test_y        

            print("done")
            
        else:
            import pandas as pd
            import numpy as np
            from PIL import Image
            from pyts.visualization import plot_gasf
            import matplotlib.pyplot as plt
            import torch
            from torch.autograd import Variable

            if is_whole==True:
                whole="_all"
            else:
                whole=""
            cross=["1","2","3","4","5"]
            cross.remove(str(cross_number_for_test))
            x = [  
                pd.read_json("ecg_normal"+whole+"_cross_"+cross[0]+".json"),
                pd.read_json("ecg_normal"+whole+"_cross_"+cross[1]+".json"),
                pd.read_json("ecg_normal"+whole+"_cross_"+cross[2]+".json"),
                pd.read_json("ecg_normal"+whole+"_cross_"+cross[3]+".json"),
                pd.read_json("ecg_abnormal"+whole+"_cross_"+cross[0]+".json"),
                pd.read_json("ecg_abnormal"+whole+"_cross_"+cross[1]+".json"),
                pd.read_json("ecg_abnormal"+whole+"_cross_"+cross[2]+".json"),
                pd.read_json("ecg_abnormal"+whole+"_cross_"+cross[3]+".json")
            ]
            x = pd.concat(x)
            x =  x.sample(frac=1)


            def pil_loader(path):
                with open(path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('L')
            lst = list()
            count=0
            lst2=list()
            y=list()

            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="ecg_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/abnormal/"
                else:
                    path="ecg_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/normal/"
                for j in x.sensor.unique():
                    img=pil_loader(path+str(i)+"_"+str(j)+".png")
                    lst.append(np.array(img)/255)

                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     count+=1
            #     if count==1:
            #         break
            train_x = torch.from_numpy(np.array(lst2)).float()
            train_y = torch.LongTensor(np.array(y))

            train_data = Dataset(train_x,train_y)
            train_loader = DataLoader(dataset=train_data,
                                      batch_size=24,
                                      shuffle=True,
                                      num_workers=2)
            self.train_x=train_x
            self.train_y=train_y
            self.train_loader=train_loader

            #test data set
            normal   = pd.read_json("ecg_normal"+whole+"_cross_"+str(cross_number_for_test)+".json")
            abnormal = pd.read_json("ecg_abnormal"+whole+"_cross_"+str(cross_number_for_test)+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            # print(x)
            lst = list()
            count=0
            lst2=list()
            y=list()
            path=""
            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="ecg_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/abnormal/"
                else:
                    path="ecg_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/normal/"

                for j in x.sensor.unique():
                    img=pil_loader(path+str(i)+"_"+str(j)+".png")
                    lst.append(np.array(img)/255)
                    
                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     if count==1:
            #         break
            test_x = Variable(torch.from_numpy(np.array(lst2)).float()).cuda()
            test_y = torch.LongTensor(np.array(y)).cuda()
            self.test_x=test_x
            self.test_y=test_y        

            print("done")
            
        
print("Function setting done")


In [ ]:
a=dataset_dataloader(True,True,"mtf")
a.train_x

## one chaneel

In [ ]:
from torch.utils.data import Dataset, DataLoader
class Dataset(Dataset):
    def __init__(self,train_x,train_y):
        self.train_x=train_x
        self.train_y=train_y
        self.data_len = len(train_x)

    def __getitem__(self, index):

        train_x=self.train_x[index]
        train_y=self.train_y[index]
        return (train_x, train_y)
    def __len__(self):
        return self.data_len
    
class dataset_dataloader:
    def __init__ (self,is_whole,is_cross,method,cross_number_for_test=1,batchsize=24,numworkers=2):
        if is_cross==False:
            import pandas as pd
            import numpy as np
            from PIL import Image
            from pyts.visualization import plot_gasf
            import matplotlib.pyplot as plt
            import torch
            from torch.autograd import Variable

            if is_whole==True:
                whole="_all"
            else:
                whole=""

            normal   = pd.read_json("ecg_normal_train"+whole+".json")
            abnormal = pd.read_json("ecg_abnormal_train"+whole+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            def pil_loader(path):
                with open(path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('L')
            lst = list()
            count=0
            lst2=list()
            y=list()

            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="ecg_img"+whole+"/"+method+"/train/abnormal/"
                else:
                    path="ecg_img"+whole+"/"+method+"/train/normal/"

                list_im = [
                    path+str(i)+'_0.png', 
                    path+str(i)+'_1.png', 
                ]
                imgs = [ pil_loader(i) for i in list_im ]
                min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
                imgs_comb = Image.fromarray( imgs_comb)
                lst.append(np.array(imgs_comb)/255)

                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     count+=1
            #     if count==1:
            #         break
            train_x = torch.from_numpy(np.array(lst2)).float()
            train_y = torch.LongTensor(np.array(y))

            train_data = Dataset(train_x,train_y)
            train_loader = DataLoader(dataset=train_data,
                                      batch_size=24,
                                      shuffle=True,
                                      num_workers=2)
            self.train_x=train_x
            self.train_y=train_y
            self.train_loader=train_loader

            #test data set
            normal   = pd.read_json("ecg_normal_validation"+whole+".json")
            abnormal = pd.read_json("ecg_abnormal_validation"+whole+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            # print(x)
            lst = list()
            count=0
            lst2=list()
            y=list()
            path=""
            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="ecg_img"+whole+"/"+method+"/validation/abnormal/"
                else:
                    path="ecg_img"+whole+"/"+method+"/validation/normal/"

                list_im = [
                    path+str(i)+'_0.png', 
                    path+str(i)+'_1.png', 

                ]
                imgs = [ pil_loader(i) for i in list_im ]
                min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
                imgs_comb = Image.fromarray( imgs_comb)
                lst.append(np.array(imgs_comb)/255)
                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     if count==1:
            #         break
            test_x = Variable(torch.from_numpy(np.array(lst2)).float()).cuda()
            test_y = torch.LongTensor(np.array(y)).cuda()
            self.test_x=test_x
            self.test_y=test_y        

            print("done")
            
        else:
            import pandas as pd
            import numpy as np
            from PIL import Image
            from pyts.visualization import plot_gasf
            import matplotlib.pyplot as plt
            import torch
            from torch.autograd import Variable

            if is_whole==True:
                whole="_all"
            else:
                whole=""
            cross=["1","2","3","4","5"]
            cross.remove(str(cross_number_for_test))
            x = [  
                pd.read_json("ecg_normal"+whole+"_cross_"+cross[0]+".json"),
                pd.read_json("ecg_normal"+whole+"_cross_"+cross[1]+".json"),
                pd.read_json("ecg_normal"+whole+"_cross_"+cross[2]+".json"),
                pd.read_json("ecg_normal"+whole+"_cross_"+cross[3]+".json"),
                pd.read_json("ecg_abnormal"+whole+"_cross_"+cross[0]+".json"),
                pd.read_json("ecg_abnormal"+whole+"_cross_"+cross[1]+".json"),
                pd.read_json("ecg_abnormal"+whole+"_cross_"+cross[2]+".json"),
                pd.read_json("ecg_abnormal"+whole+"_cross_"+cross[3]+".json")
            ]
            x = pd.concat(x)
            x =  x.sample(frac=1)


            def pil_loader(path):
                with open(path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('L')
            lst = list()
            count=0
            lst2=list()
            y=list()

            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="ecg_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/abnormal/"
                else:
                    path="ecg_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/normal/"

                list_im = [
                    path+str(i)+'_0.png', 
                    path+str(i)+'_1.png', 
                ]
                imgs = [ pil_loader(i) for i in list_im ]
                min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
                imgs_comb = Image.fromarray( imgs_comb)
                lst.append(np.array(imgs_comb)/255)

                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     count+=1
            #     if count==1:
            #         break
            train_x = torch.from_numpy(np.array(lst2)).float()
            train_y = torch.LongTensor(np.array(y))

            train_data = Dataset(train_x,train_y)
            train_loader = DataLoader(dataset=train_data,
                                      batch_size=24,
                                      shuffle=True,
                                      num_workers=2)
            self.train_x=train_x
            self.train_y=train_y
            self.train_loader=train_loader

            #test data set
            normal   = pd.read_json("ecg_normal"+whole+"_cross_"+str(cross_number_for_test)+".json")
            abnormal = pd.read_json("ecg_abnormal"+whole+"_cross_"+str(cross_number_for_test)+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            # print(x)
            lst = list()
            count=0
            lst2=list()
            y=list()
            path=""
            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="ecg_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/abnormal/"
                else:
                    path="ecg_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/normal/"

                list_im = [
                    path+str(i)+'_0.png', 
                    path+str(i)+'_1.png', 
                ]
                imgs = [ pil_loader(i) for i in list_im ]
                min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
                imgs_comb = Image.fromarray( imgs_comb)
                lst.append(np.array(imgs_comb)/255)
                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     if count==1:
            #         break
            test_x = Variable(torch.from_numpy(np.array(lst2)).float()).cuda()
            test_y = torch.LongTensor(np.array(y)).cuda()
            self.test_x=test_x
            self.test_y=test_y        

            print("done")
            
        
print("Function setting done")


# <font color="#f47641">Cnn start</font>  

In [ ]:
### import torch.nn as nn
import torch
from torch.autograd import Variable
accuracy_matrix =[]
import dataset
import dataset_six

    
for i in range(5):
    min_error=100
    min_lose=100
    count=0
    number=1
#     cross=dataset.dataset_dataloader(False,True,"mtf",i+1)
    cross=dataset_dataloader(True,True,"mtf",i+1)
    import torch.nn as nn

    EPOCH = 50              # train the training data n times, to save time, we just train 1 epoch
    LR = 0.0023
    
    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 =nn.Sequential( #卷基層   
                nn.Conv2d(
                    in_channels  = 2  , # 圖片是有幾層的 若 RGB三層 灰階 1層
                    out_channels = 10, # 同時16個filter 進行掃描 會提取16個特徵 代表下一層高度為16
                    kernel_size  = 5  , # 一次畫出來的框 畫幾格 ex 5*5
                    stride       = 1  , # 每一個框框跳幾格
                    padding      = 2, # 在 5x5逐步掃描後會有少的格子 因此2代表在外面為兩圈的0
                                        # if stride =1 padding = (kernel_size-1)/2
                ),  # 過濾器 卷基層 蒐集圖片訊息 三維的空間  
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2), #找出更重要的特徵  像是在2x2中 找到 最大的值
            )
            self.conv2 = nn.Sequential(
                nn.Conv2d(10,20,5,1,2), # 輸入的是上一層的16 把輸出層在變大因此是32 其他不變
                nn.ReLU(),
                nn.MaxPool2d(2)
            )
#             self.conv3 = nn.Sequential(
#                 nn.Conv2d(5,10,5,1,2), # 輸入的是上一層的16 把輸出層在變大因此是32 其他不變
#                 nn.ReLU(),
#                 nn.MaxPool2d(2)
#             )

            self.out = nn.Linear(20*32*32,2)
        def forward(self,x):

            x=self.conv1(x)
            x=self.conv2(x) #(batch.32.7.7)
#             x=self.conv3(x)
            x=x.view(x.size(0),-1) #將三維轉二維 (batch , 32*7*7)
            output=self.out(x)
            return output,x

    cnn=CNN()
    cnn.cuda()
#     print(cnn)

    optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
    loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

    # following function (plot_with_labels) is for visualization, can be ignored if not interested

    # training and testing


    for epoch in range(EPOCH):
        for step, (x, y) in enumerate(cross.train_loader):   # gives batch data, normalize x when iterate train_loader
            b_x = Variable(x).cuda()   # batch x
            b_y = Variable(y).cuda()   # batch y

            output = cnn(b_x)[0]             # cnn output
            loss = loss_func(output, b_y)   # cross entropy loss
            optimizer.zero_grad()           # clear gradients for this training step
            loss.backward()                 # backpropagation, compute gradients
            optimizer.step()                # apply gradients
            test_output, last_layer = cnn(cross.test_x)
            pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()
            accuracy = sum(pred_y == cross.test_y) / float(cross.test_y.size(0))
            error  = (1-accuracy)*100
            if error<min_error:
                min_error=error
                min_lose=100
                count=0
            else:
                count+=1
                print(".",end="")

            if error == min_error:
                if loss.data[0]<min_lose:
                    min_lose=loss.data[0]
                    torch.save(cnn,'cnn_save/cnn'+str(number)+'.pkl')
                    number+=1
    #                 print('save!!')
    #                 print('\ncount: ',count,'Epoch: ', epoch, '| train loss: %.15f' % loss.data[0], '| min validation error rate: %.20f' % min_error)
            if count>=350:
                break
        if count>=350:
            break
    
    # print 10 predictions from test data
    # test_output, _ = cnn(test_x)
    # pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()

    # print(pred_y, 'prediction number')
    # print(test_y, 'real number')
    accuracy_matrix.append(min_error)
    print('\n number: ',number-1 ,min_error)
    
import IPython.display as ipd
import numpy
sr = 22050 # sample rate
T = 0.3    # seconds
t = numpy.linspace(0, T, int(T*sr), endpoint=False) # time variable
x = 0.5*numpy.sin(2*numpy.pi*490*t)                # pure sine wave at 440 284Hz
ipd.Audio(x, rate=sr,autoplay=True) # load a NumPy array


In [ ]:
print(accuracy_matrix)
sum(accuracy_matrix) / float(len(accuracy_matrix))

In [ ]:
### import torch.nn as nn
import torch
from torch.autograd import Variable
accuracy_matrix =[]
import dataset
import dataset_six

    
for i in range(5):
    min_error=100
    min_lose=100
    count=0
    number=1
#     cross=dataset.dataset_dataloader(False,True,"mtf",i+1)
    cross=dataset_dataloader(True,True,"mtf",i+1)
    import torch.nn as nn

    EPOCH = 50              # train the training data n times, to save time, we just train 1 epoch
    LR = 0.0023
    
    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 =nn.Sequential( #卷基層   
                nn.Conv2d(
                    in_channels  = 1  , # 圖片是有幾層的 若 RGB三層 灰階 1層
                    out_channels = 3, # 同時16個filter 進行掃描 會提取16個特徵 代表下一層高度為16
                    kernel_size  = 5  , # 一次畫出來的框 畫幾格 ex 5*5
                    stride       = 1  , # 每一個框框跳幾格
                    padding      = 2, # 在 5x5逐步掃描後會有少的格子 因此2代表在外面為兩圈的0
                                        # if stride =1 padding = (kernel_size-1)/2
                ),  # 過濾器 卷基層 蒐集圖片訊息 三維的空間  
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2), #找出更重要的特徵  像是在2x2中 找到 最大的值
            )
            self.conv2 = nn.Sequential(
                nn.Conv2d(3,5,5,1,2), # 輸入的是上一層的16 把輸出層在變大因此是32 其他不變
                nn.ReLU(),
                nn.MaxPool2d(2)
            )
#             self.conv3 = nn.Sequential(
#                 nn.Conv2d(5,10,5,1,2), # 輸入的是上一層的16 把輸出層在變大因此是32 其他不變
#                 nn.ReLU(),
#                 nn.MaxPool2d(2)
#             )

            self.out = nn.Linear(5*64*32,2)
        def forward(self,x):

            x=self.conv1(x)
            x=self.conv2(x) #(batch.32.7.7)
#             x=self.conv3(x)
            x=x.view(x.size(0),-1) #將三維轉二維 (batch , 32*7*7)
            output=self.out(x)
            return output,x

    cnn=CNN()
    cnn.cuda()
#     print(cnn)

    optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
    loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

    # following function (plot_with_labels) is for visualization, can be ignored if not interested

    # training and testing


    for epoch in range(EPOCH):
        for step, (x, y) in enumerate(cross.train_loader):   # gives batch data, normalize x when iterate train_loader
            b_x = Variable(x).cuda()   # batch x
            b_y = Variable(y).cuda()   # batch y

            output = cnn(b_x)[0]             # cnn output
            loss = loss_func(output, b_y)   # cross entropy loss
            optimizer.zero_grad()           # clear gradients for this training step
            loss.backward()                 # backpropagation, compute gradients
            optimizer.step()                # apply gradients
            test_output, last_layer = cnn(cross.test_x)
            pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()
            accuracy = sum(pred_y == cross.test_y) / float(cross.test_y.size(0))
            error  = (1-accuracy)*100
            if error<min_error:
                min_error=error
                min_lose=100
                count=0
            else:
                count+=1
                print(".",end="")

            if error == min_error:
                if loss.data[0]<min_lose:
                    min_lose=loss.data[0]
                    torch.save(cnn,'cnn_save/cnn'+str(number)+'.pkl')
                    number+=1
    #                 print('save!!')
    #                 print('\ncount: ',count,'Epoch: ', epoch, '| train loss: %.15f' % loss.data[0], '| min validation error rate: %.20f' % min_error)
            if count>=350:
                break
        if count>=350:
            break
    
    # print 10 predictions from test data
    # test_output, _ = cnn(test_x)
    # pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()

    # print(pred_y, 'prediction number')
    # print(test_y, 'real number')
    accuracy_matrix.append(min_error)
    print('\n number: ',number-1 ,min_error)
    
import IPython.display as ipd
import numpy
sr = 22050 # sample rate
T = 0.3    # seconds
t = numpy.linspace(0, T, int(T*sr), endpoint=False) # time variable
x = 0.5*numpy.sin(2*numpy.pi*490*t)                # pure sine wave at 440 284Hz
ipd.Audio(x, rate=sr,autoplay=True) # load a NumPy array


In [ ]:
print(accuracy_matrix)
sum(accuracy_matrix) / float(len(accuracy_matrix))